taken from gmacgilchrist/cmip6/dev_cmip6.ipynb

In [8]:
import xarray as xr
from cmip6_preprocessing.preprocessing import combined_preprocessing
# import PyCO2SYS as pyco2 # ModuleNotFoundError: No module named 'PyCO2SYS'
import intake
import matplotlib.pyplot as plt
import numpy as np
import warnings
import gsw
warnings.filterwarnings("ignore")

In [9]:
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col = intake.open_esm_datastore(url)
col.df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,huss,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,hus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


In [10]:
col.df.tail()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
501651,LS3MIP,RUBISCO,E3SM-1-1,land-hist,r1i1p11f1,Amon,tas,gn,gs://cmip6/CMIP6/LS3MIP/RUBISCO/E3SM-1-1/land-...,NaN,20210507
501652,LS3MIP,RUBISCO,E3SM-1-1,land-hist,r1i1p11f1,Emon,mrsol,gn,gs://cmip6/CMIP6/LS3MIP/RUBISCO/E3SM-1-1/land-...,NaN,20210507
501653,LS3MIP,RUBISCO,E3SM-1-1,land-hist,r1i1p11f1,Lmon,cLitter,gn,gs://cmip6/CMIP6/LS3MIP/RUBISCO/E3SM-1-1/land-...,NaN,20210507
501654,LS3MIP,RUBISCO,E3SM-1-1,land-hist,r1i1p11f1,Lmon,cVeg,gn,gs://cmip6/CMIP6/LS3MIP/RUBISCO/E3SM-1-1/land-...,NaN,20210507
501655,LS3MIP,RUBISCO,E3SM-1-1,land-hist,r1i1p11f1,Lmon,mrro,gn,gs://cmip6/CMIP6/LS3MIP/RUBISCO/E3SM-1-1/land-...,NaN,20210507


In [11]:
# what i think this is doing is just identifying all the unique source_IDs, 
# which i think is the name of each model
col.unique(columns='source_id')

{'source_id': {'count': 86,
  'values': ['ECMWF-IFS-LR',
   'BCC-CSM2-HR',
   'MPI-ESM1-2-LR',
   'MIROC-ES2L',
   'CMCC-CM2-HR4',
   'CESM2-WACCM',
   'TaiESM1',
   'CAS-ESM2-0',
   'CAMS-CSM1-0',
   'KACE-1-0-G',
   'MCM-UA-1-0',
   'ACCESS-CM2',
   'ACCESS-ESM1-5',
   'GFDL-AM4',
   'FGOALS-g3',
   'MRI-ESM2-0',
   'HadGEM3-GC31-MM',
   'CIESM',
   'NESM3',
   'CNRM-CM6-1',
   'BCC-ESM1',
   'AWI-ESM-1-1-LR',
   'CESM1-1-CAM5-CMIP5',
   'GISS-E2-1-H',
   'CESM2-FV2',
   'NorESM2-MM',
   'EC-Earth3-AerChem',
   'FGOALS-f3-H',
   'E3SM-1-1',
   'EC-Earth3-CC',
   'MIROC-ES2H',
   'CMCC-CM2-VHR4',
   'CESM1-WACCM-SC',
   'MIROC6',
   'EC-Earth3',
   'GFDL-OM4p5B',
   'CNRM-ESM2-1',
   'KIOST-ESM',
   'EC-Earth3-Veg-LR',
   'MRI-AGCM3-2-S',
   'INM-CM4-8',
   'NorESM2-LM',
   'CESM2-WACCM-FV2',
   'EC-Earth3-LR',
   'CanESM5',
   'INM-CM5-0',
   'CNRM-CM6-1-HR',
   'GFDL-ESM4',
   'MPI-ESM1-2-XR',
   'CMCC-ESM2',
   'CMCC-CM2-SR5',
   'NorCPM1',
   'MPI-ESM1-2-HR',
   'GFDL-ESM2M',
   '

In [12]:
# giving me the unique variables for GFDL-CM4, CMIP, pre-industrial control, for monthly ocean and yearly ocean
col.search(source_id='GFDL-CM4',activity_id='CMIP',experiment_id='piControl',table_id=['Omon','Oyr']).unique(columns='variable_id')
# presumably I can do this for every model i need to identify some v

{'variable_id': {'count': 56,
  'values': ['bddtdic',
   'tos',
   'ph',
   'bddtdife',
   'co3satcalc',
   'o2',
   'phyc',
   'mlotstmin',
   'thetao',
   'volcello',
   'zos',
   'expcalc',
   'masso',
   'msftyz',
   'tob',
   'graz',
   'zostoga',
   'volo',
   'mlotst',
   'msftyzsmpa',
   'phyp',
   'pbfe',
   'remoc',
   'dissic',
   'expc',
   'pcalc',
   'pso',
   'po4',
   'co3',
   'dissoc',
   'bddtdip',
   'sob',
   'wfo',
   'pp',
   'mlotstmax',
   'friver',
   'expp',
   'fescav',
   'dfe',
   'pbo',
   'evs',
   'bddtalk',
   'chl',
   'zossq',
   'hfds',
   'soga',
   'msftyrho',
   'dcalc',
   'mfo',
   'wmo',
   'sosga',
   'sos',
   'so',
   'expfe',
   'thetaoga',
   'talk']}}

dissic: dissolved inorganic carbon
thetao: potential temp
so: salinity

In [ ]:
variables = ['dissic','talk','so','thetao']
z_kwargs = {'consolidated': True, 'decode_times':False}
query = dict(experiment_id=['piControl'], table_id=['Omon'], 
             variable_id=variables,
             grid_label=['gn','gr'],
             source_id=['CanESM5-CanOE', 'CanESM5', 'ACCESS-ESM1-5',
                        'MPI-ESM-1-2-HAM', 'IPSL-CM6A-LR', 'MIROC-ES2L',
                        'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'NorCPM1',
                        'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM', 'GFDL-ESM4'])
cat = col.search(**query)
# print(cat.df['source_id'].unique())
dset_dict = cat.to_dataset_dict(zarr_kwargs=z_kwargs,
                                preprocess=combined_preprocessing)

temp={}
for name,item in dset_dict.items():
#     print(name)
#     print(item.data_vars)
    present = item.data_vars
    if all(i in present for i in variables):
        temp[name]=dset_dict[name]
dset_dict = temp
dset_dict


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
MIROC-ES2L: No units found
